In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import unicodedata
import logging
import csv
import time
import re
import json
import pandas as pd
import mysql.connector
db = mysql.connector.connect(
  host="localhost",
  user="root",
  password="Jzy969811",
  database="movie_sys"
)

cursor = db.cursor()
movie_meta_path = "raw_data/movies_metadata.csv"
movie_staff_path = "raw_data/credits.csv"
movie_keyword_path = "raw_data/keywords.csv"
movie_id_path = "raw_data/links.csv"
movie_merged_info_path = "info/movie_info.csv"
movie_extra_info_path = "info/extra_info.csv"
movie_final_info_path = "info/movie_final_info.csv"

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
df_meta = pd.read_csv(movie_meta_path)[["genres", "id", "spoken_languages", "title", "overview","release_date","imdb_id"]]
df_meta = df_meta.fillna(value="")
for index, row in df_meta.iterrows():
    if row['imdb_id']:
        df_meta.loc[index, 'imdb_id'] = int(row['imdb_id'][2:])
    if row['release_date'] != "":
        if row['release_date'].find("-") == -1:
            df_meta.loc[index, 'release_date'] = datetime.strptime(row['release_date'], '%Y/%m/%d').strftime('%Y-%m-%d')
    df_meta.loc[index,"id"] = int(df_meta["id"][index])
    genre = row['genres'].replace("'", '"')
    language = row['spoken_languages'].replace("'", '"')
    language = re.sub(r'\\','',language)
    df_meta.loc[index,"genres"] = ",".join([item['name'] for item in json.loads(genre)])
    if language:
        df_meta.loc[index,"spoken_languages"] = ",".join([item['name'] for item in json.loads(language)])
df_meta.rename(columns={'imdb_id':'imdbId'}, inplace=True)
df_meta.rename(columns={'id':'tmdbId'}, inplace=True)

In [3]:
df_kw = pd.read_csv(movie_keyword_path)
df_kw = df_kw.fillna(value="")
for index, row in df_kw.iterrows():
    keywords = row['keywords']
    keywords = keywords.replace("\\","/")
    words = keywords.split("}, {")
    res = ""
    for i,word in enumerate(words):
        words[i] = re.sub(r"'id'", r'"id"',word)
        words[i] = re.sub(r"'name'", r'"name"',words[i])
        words[i] = re.sub(r": '(.*)'", r': "\1"', words[i])
        res += f"{words[i]}" 
        if i != len(words) - 1:
            res += "}, {"
    try:
        df_kw.loc[index,"keywords"] = ",".join([item['name'] for item in json.loads(res)])
    except Exception as e:
        print(res)
df_kw.rename(columns={'id':'tmdbId'}, inplace=True)

In [4]:
df_stf = pd.read_csv(movie_staff_path)[["id","cast"]]
df_stf = df_stf.fillna(value="")
for index, row in df_stf.iterrows():
    cast = row['cast']
    cast = cast.replace("\\","/")
    cast = re.sub(r"None", r'"None"',cast)
    casts = cast.split("'name': ")
    res_cast = []
    for i in range(1,len(casts)):
        if re.search(r"'(.*?)'", casts[i]):
            res_cast.append(re.search(r"'(.*?)'", casts[i]).group(1))
    df_stf.loc[index,"cast"] = ",".join(res_cast)   
df_stf.rename(columns={'id':'tmdbId'}, inplace=True)

In [5]:
df_id = pd.read_csv(movie_id_path)[['tmdbId','imdbId']]

In [6]:
df_meta = df_meta.drop_duplicates(subset=['imdbId','tmdbId'])
df_id = df_id.drop_duplicates(subset=['imdbId','tmdbId'])
df_kw = df_kw.drop_duplicates(subset='tmdbId')
df_stf = df_stf.drop_duplicates(subset='tmdbId')
df1 = pd.merge(df_meta, df_id, on=['imdbId','tmdbId'])
df2 = pd.merge(df1, df_stf, on='tmdbId')
df_bs = pd.merge(df2, df_kw, on='tmdbId')
df_bs.to_csv(movie_merged_info_path,index=False)

In [7]:
df_ex = pd.read_csv(movie_extra_info_path,header=None).rename(columns={0:'imdbId',1:"director",2:"poster"})
df_bs = pd.read_csv(movie_merged_info_path)
df_final = pd.merge(df_ex, df_bs, on='imdbId')
df_final.to_csv(movie_final_info_path,index=False)

In [8]:
df_final = pd.read_csv(movie_final_info_path)
add_data = ("INSERT INTO MovieInfo "
            "(tmdb_id, imdb_id, movie_name, spoken_language, director, outline, poster, actor, keyword, genre, publish_date) "
            "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)")

for i, r in df_final.iterrows():
    data = (r["tmdbId"],r["imdbId"],r["title"],r["spoken_languages"],r["director"],r["overview"],r["poster"],r["cast"],r["keywords"],r["genres"],r["release_date"])
    cursor.execute(add_data, data)

db.commit()
cursor.close()
db.close()